# Ascaso-Extended Models Calibration


---
**License**

 Ascaso_Extended_Calibration

 Tue Aug 24 10:30:00 2021\
 Copyright  2021\
 Cinthia Nunes de Lima <cinthia.n.lima@uel.br> \ Sandro Dias Pinto Vitenti <vitenti@uel.br>

---
---

 Ascaso_Extended_Calibration\
 Copyright (C) 2021 Cinthia Nunes de Lima <cinthia.n.lima@uel.br>, Sandro Dias Pinto Vitenti <vitenti@uel.br>

 numcosmo is free software: you can redistribute it and/or modify it
 under the terms of the GNU General Public License as published by the
 Free Software Foundation, either version 3 of the License, or
 (at your option) any later version.

 numcosmo is distributed in the hope that it will be useful, but
 WITHOUT ANY WARRANTY; without even the implied warranty of
 MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.
 See the GNU General Public License for more details.

 You should have received a copy of the GNU General Public License along
 with this program.  If not, see <http://www.gnu.org/licenses/>.
 
---

## Packages

In [1]:
import sys
sys.path.insert(0, "/global/homes/c/cinlima/NumCosmo/notebooks/richness_proxy/")
sys.path.insert(0, "/global/homes/c/cinlima/NumCosmo/notebooks/richness_proxy/Scripts/")
sys.path.insert(0, "/global/homes/c/cinlima/NumCosmo/notebooks/richness_proxy/CatalogsMatching")


#NumCosmo
from numcosmo_py import Ncm, Nc, GObject
from richness_mass_calib import create_richness_mass_calib

#Useful packages
import numpy as np
import math
from scipy import stats

from astropy.io import fits
from astropy.table import Table
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error

import os

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.pyplot as gridspec
from matplotlib.gridspec import GridSpec

sys.path.insert(0, "/global/homes/c/cinlima//gcr-catalogs")
import GCRCatalogs

%matplotlib inline

Ncm.cfg_init()
Ncm.cfg_set_log_handler(lambda msg: sys.stdout.write(msg) and sys.stdout.flush())

sys.path.insert(0, "global/homes/c/cinlima//clevar/clevar")

#Put nerc if using nersc
GCRCatalogs.set_root_dir_by_site("nersc")


# import clevar
# from clevar import ClCatalog, MemCatalog
# from clevar.match import MembershipMatch, get_matched_pairs
# from clevar.match import output_matched_catalog
# from clevar.match_metrics.scaling import ClCatalogFuncs as s_cf


#Calibration functions
from bdata import DataB
from model_fitting import ModelFitting

## Data

### Halos m200c

In [2]:
# DC2_halos_m200c = fits.open(
#     "/global/cfs/projectdirs/lsst/groups/CL/cosmoDC2_v1.1.4/extragal/full/halos/halos_m200c_13.0.fits")

# dt_halos = Table(DC2_halos_m200c[1].data)
# dt_halos = dt_halos[dt_halos["redshift_true"] < 1.2]

# lnM = np.log(dt_halos["m200c"])  # ln(mass)
# z = dt_halos["redshift_true"]  # redshift
# lnR = np.log(dt_halos["richness"])  # ln(richness)


### Clevar

In [3]:
# CLEVAR

# min_richness = 5
# min_halo_mass = 1e13 #Msun

# #Halos/Clusters
# #SkySim + RedMaPPer: 'skysim5000_v1.1.1_redmapper_v0.8.5'
# RM_cat = ClCatalog.read('/global/homes/c/cinlima/NumCosmo/notebooks/richness_proxy/SSData/RM_cat_richness_min_'+str(min_richness)+'.fits', name='RM_cat', full=True, tags={'mass':'richness'})

# #SkySim: 'skysim5000_v1.1.1_image'
# halo_cat = ClCatalog.read('/global/homes/c/cinlima/NumCosmo/notebooks/richness_proxy/SSData/halo_cat_mass_min_'+f"{min_halo_mass:1.2e}"+'.fits', name='halo_cat', full=True, tags={'mass':'mass_fof'})

# #Members
# RM_cat.read_members('/global/homes/c/cinlima/NumCosmo/notebooks/richness_proxy/SSData/RM_cat_richness_min_'+str(min_richness)+'_members.fits',  full=True)
# halo_cat.read_members('/global/homes/c/cinlima/NumCosmo/notebooks/richness_proxy/SSData/halo_cat_mass_min_'+f"{min_halo_mass:1.2e}"+'_members.fits', full=True)

# #Match catalogs
# mt = MembershipMatch()

# mt.load_matches(RM_cat, halo_cat, out_dir='/global/homes/c/cinlima/NumCosmo/notebooks/richness_proxy/SSData/SS_RM_match_cross_membership')

# info = s_cf.plot(RM_cat, halo_cat, 'cross', col='mass', xscale='log', yscale='log',
#                  col_color='z', add_err=False)

In [4]:
# Halos/Clusters
# #CosmoDC2 + RedMaPPer: 'cosmoDC2_v1.1.4_redmapper_v0.8.1'
# RM_cat = ClCatalog.read('/global/homes/c/cinlima/NumCosmo/notebooks/richness_proxy/CDC2Data/RM_cat_richness_min_'+str(min_richness)+'.fits', name='RM_cat', full=True, tags={'mass':'richness'})

# #CosmoDC2: 'cosmoDC2_v1.1.4_image'
# halo_cat = ClCatalog.read('/global/homes/c/cinlima/NumCosmo/notebooks/richness_proxy/CDC2Data/halo_cat_mass_min_'+f"{min_halo_mass:1.2e}"+'.fits', name='halo_cat', full=True, tags={'mass':'mass_fof'})

# #Members
# RM_cat.read_members('/global/homes/c/cinlima/NumCosmo/notebooks/richness_proxy/CDC2Data/RM_cat_richness_min_'+str(min_richness)+'_members.fits',  full=True)
# halo_cat.read_members('/global/homes/c/cinlima/NumCosmo/notebooks/richness_proxy/CDC2Data/halo_cat_mass_min_'+f"{min_halo_mass:1.2e}"+'_members.fits', full=True)

# #Match catalogs
# mt = MembershipMatch()

# mt.load_matches(RM_cat, halo_cat, out_dir='/global/homes/c/cinlima/NumCosmo/notebooks/richness_proxy/CDC2Data/cosmoDC2_RM_match_cross_membership')


In [5]:
# RM_cat_bij, halo_cat_bij = get_matched_pairs(RM_cat, halo_cat, 'cross')

# clusters_dt = Table([halo_cat_bij['mass_fof'], RM_cat_bij['richness'], RM_cat_bij['z']], names=('m200c', 'richness', 'redshift_true'))
# dt_halos = clusters_dt[np.logical_and(clusters_dt['redshift_true'] > 0.14, clusters_dt['redshift_true']< 0.33)]
# dt_halos = dt_halos[dt_halos['richness'] > 20]
# dt_halos = clusters_dt

In [6]:
# cosmodc2= GCRCatalogs.load_catalog('cosmoDC2_v1.1.4_small')


### NumCosmo's Matching By Proximity

In [7]:
# mdata = Table.read('CatalogsMatching/test.fits')
# data = mdata[[len(m) > 0 for m in mdata['mass']]]

# mass_data= [m[0] for m in data['mass']]
# rich_data = [r for r in data['richness']]
# z_data = [z for z in data['z']]

# test_dt = Table([mass_data, rich_data, z_data], names=('mass', 'richness', 'redshift'))



In [8]:
# lnM_binned = bd.get_lnM_binned()
# lnR_binned = bd.get_lnR_binned()
# z_binned = bd.get_z_binned()
# bins_std = bd.get_bins_std()
# bins_mean = bd.get_bins_mean()

### Matching by ID without Clevar

In [9]:
# test_matchdata = pd.read_csv('/global/homes/c/cinlima/MatchingCatalogs/match_catalog.csv',  sep = ';')
test_matchdata = Table.read('CatalogsMatching/match_ID.fits')

rich_data = test_matchdata["richness"]
z_data = test_matchdata["redshift"]
mass_data = test_matchdata["halo_mass"]
r_err_data = test_matchdata["richness_err"]

test_dt = Table([mass_data, rich_data, z_data, r_err_data], names=('mass', 'richness', 'redshift', 'richness_err'))

df_test_matchdata = test_matchdata.to_pandas()
# df_test_matchdata

data_df = test_dt

### Data cuts

In [10]:
# #DATA

# fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10,4))

# ax1.loglog(mass_data, rich_data, '.', c = 'k')
# ax1.set(xlabel='$\ln M$', ylabel='$\ln \lambda$')

# ax2.plot(z_data, rich_data, '.', c = 'k')
# ax2.set(xlabel='z', ylabel='$\ln \lambda$')


In [11]:
# #DATA

# fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10,4))

# ax1.plot(mass_data, r_err_data, '.', c = 'k')
# ax1.set(xlabel='$\ln M$', ylabel='Err $\ln \lambda$')

# ax2.plot(z_data, r_err_data, '.', c = 'k')
# ax2.set(xlabel='z', ylabel='Err $\ln \lambda$')

In [12]:
# # Richness_err Cuts 

# fig, axs = plt.subplots(nrows=2, ncols=2,figsize=(10,8))

# errcut_list = np.array([1.0, 1.5, 2.0, 5.0])

# for errcut, ax in zip(errcut_list, axs.flat):

#     test_dt = Table([mass_data, rich_data, z_data, r_err_data], names=('mass', 'richness', 'redshift', 'richness_err'))

#     test_dt = test_dt[test_dt['richness_err'] < errcut]

#     mass_dt = test_dt['mass']
#     red_dt = test_dt['redshift']
#     rich_dt = test_dt['richness']
        
#     ax.scatter(np.log(mass_dt), np.log(rich_dt), c='k')
#     ax.set(xlabel='lnM', ylabel='ln$\lambda$')
#     ax.set_title('Richness error cut: $\lambda$ error <' +str(errcut) )

#     fig.tight_layout()
    

In [13]:
# # Richness Cuts 

# fig, axs = plt.subplots(nrows=2, ncols=2,figsize=(10,8))

# rcut_list = np.array([10, 20, 30, 50])
# for rcut, ax in zip(rcut_list, axs.flat):
    
#     test_dt = Table([mass_data, rich_data, z_data], names=('mass', 'richness', 'redshift'))

#     test_dt = test_dt[np.logical_and(test_dt['richness'] > rcut, test_dt['richness'] < 1e3)]
#     # test_dt = test_dt[np.logical_and(test_dt['mass'] > 1e14, test_dt['mass'] < 1e16)]


#     mass_dt = test_dt['mass']
#     red_dt = test_dt['redshift']
#     rich_dt = test_dt['richness']
        
#     ax.scatter(np.log(mass_dt), np.log(rich_dt), c='k')
#     ax.set(xlabel='lnM', ylabel='ln$\lambda$')
#     ax.set_title('Richness cut: $\lambda >$' +str(rcut) )

#     fig.tight_layout()
    

In [14]:
# fig, axs = plt.subplots(nrows=2, ncols=2,figsize=(10,8))

# mcut_list = np.array([1e13, 0.2e14, 0.5e14, 1e14])
# for mcut, ax in zip(mcut_list, axs.flat):
    
#     test_dt = Table([mass_data, rich_data, z_data], names=('mass', 'richness', 'redshift'))

#     test_dt = test_dt[np.logical_and(test_dt['mass'] > mcut, test_dt['mass'] < 1e16)]
   
#     ax.plot(np.log(test_dt['mass']), np.log(test_dt['richness']),'.', c='k')
#     ax.set(xlabel='ln(M)_mean', ylabel='$\ln \lambda$')
#     ax.set_title('Mass cut: $M >$' +'{:.2e}'.format(mcut))

#     fig.tight_layout()

In [15]:
# # Data threshold
# test_dt = Table([mass_data, rich_data, z_data, r_err_data], names=('mass', 'richness', 'redshift', 'richness_err'))
# test_dt = test_dt[test_dt['mass'] > 5.0e13]
# test_dt = test_dt[np.logical_and(test_dt['redshift'] > 0, test_dt['redshift']< 3)]
# test_dt = test_dt[np.logical_and(test_dt['richness'] > 0, test_dt['richness'] < 1e3)]

# mass_dt = test_dt['mass']
# red_dt = test_dt['redshift']
# rich_dt = test_dt['richness']

# print('\t\tMin:\t\t\tMax:')
# print(f'Mass:\t\t{min(mass_dt):.2e},\t\t{max(mass_dt):.2e}')
# print(f'Redshift:\t{min(red_dt)},\t{max(red_dt)}')
# print(f'Richness:\t{min(rich_dt)},\t{max(rich_dt)}')


# dt_halos = test_dt

## Models fit

### Training and test sets

In [16]:
# #Length of bins
# zl = 0.05
# ml = 0.05
# bd = DataB(test_dt, zl, ml)

In [17]:
# X = pd.DataFrame({'mass': list(dt_halos["mass"]), 'redshift': list(dt_halos["redshift"])})
# y = pd.DataFrame({'richness': list(dt_halos["richness"])})

# fm = FittingModel(dt_halos) #Fit object

# data_train = fm.cvdata(X,y)[0]
# data_test = fm.cvdata(X,y)[1]

#Test  data
# data_test = pd.concat([X_test, y_test], axis=1)

# print( data_test.shape, X_test.shape, y_test.shape )
# print( data_train.shape, X_train.shape, y_train.shape )

# print( data_test.shape[0] + data_train.shape[0] )

# print( len(dt_halos) )


### Bins


In [18]:
# # Bins length
# halos_mean = bd.get_bins_mean()
# halos_std = bd.get_bins_std()
# # print(f'Total bins in halos_bin_mz: {len(hbmz)}')
# print(f'Total bins in halos_mean: {len(bd.get_bins_mean())}')


In [19]:
# fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10,4))

# ax1.plot(np.log(halos_mean['mass']), np.log(halos_mean['richness']),'.', c='k')
# ax1.set(xlabel='ln(M)_mean', ylabel='$\mu_{\ln \lambda}$')

# ax2.plot(halos_mean['redshift'], np.log(halos_mean['richness']),'.', c='k')
# ax2.set(xlabel='z_mean', ylabel='$\mu_{\ln \lambda}$')

# plt.suptitle('$\mu_{\ln \lambda}$')
# plt.show()


In [20]:
# fig, axs = plt.subplots(nrows=2, ncols=2,figsize=(10,8))

# rcut_list = np.array([10, 20, 30, 50])
# for rcut, ax in zip(rcut_list, axs.flat):
    
#     test_dt = Table([mass_dt, rich_dt, red_dt, dt_halos['richness_err']], names=('mass', 'richness', 'redshift', 'richness_err'))

#     test_dt = test_dt[np.logical_and(test_dt['richness'] > rcut, test_dt['richness'] < 1e3)]

#     bd = DataB(test_dt, zl, ml)
#     halos_mean = bd.get_bins_mean()
        
#     ax.plot(np.log(halos_mean['mass']), np.log(halos_mean['richness']),'.', c='k')
#     ax.set(xlabel='ln(M)_mean', ylabel='$\mu_{\ln \lambda}$')
#     ax.set_title('Richness cut: $\lambda >$' +str(rcut) )

#     fig.tight_layout()
    


In [21]:
# fig, axs = plt.subplots(nrows=2, ncols=2,figsize=(10,8))

# mcut_list = np.array([1e13, 0.2e14, 0.5e14, 1e14])
# for mcut, ax in zip(mcut_list, axs.flat):
    
#     test_dt = Table([mass_dt, rich_dt, red_dt, dt_halos['richness_err']], names=('mass', 'richness', 'redshift', 'richness_err'))

#     test_dt = test_dt[np.logical_and(test_dt['mass'] > mcut, test_dt['mass'] < 1e16)]

#     bd = DataB(test_dt, zl, ml)
#     halos_mean = bd.get_bins_mean()
        
#     ax.plot(np.log(halos_mean['mass']), np.log(halos_mean['richness']),'.', c='k')
#     ax.set(xlabel='ln(M)_mean', ylabel='$\mu_{\ln \lambda}$')
#     ax.set_title('Mass cut: $M >$' +'{:.2e}'.format(mcut))

#     fig.tight_layout()

### fit

In [22]:
bd2 = DataB(data_df, 0.05, 0.05)
fm = FittingModel(data_df)

halos_mean = bd2.get_bins_mean()
halos_std = bd2.get_bins_std()


NameError: name 'FittingModel' is not defined

### Ascaso Fit Params

Here we do the fitting of linear model parameters using training data in **data_train**. The model is:

\begin{equation}
    \left< \ln R | M, z \right> = \mu_0 + \mu_1 \ln\left( \frac{M}{M_0} \right) +  \mu_2\ln\left(\frac{1 + z}{1 + z_0}\right);
\end{equation}

$$\textbf{p}_{1} = \{\mu_0, \mu_1, \mu_2, \sigma_0, \sigma_1, \sigma_2\}. $$

$\textbf{p}_{1}$ is the vector with the linear model parameters.


In [ ]:
ascaso, fit_ascaso = fm.run_fit('ascaso')

In [ ]:
lnR_mean_ascaso = fm.get_mean_model(ascaso, np.log(halos_mean['mass']),  halos_mean['redshift'])
lnR_std_ascaso = fm.get_std_model(ascaso, np.log(halos_mean['mass']), halos_mean['redshift'])

### Ext (ln(1+z)) Fit Params
Here we do the fitting of quadratic model parameters using training data in **data_train**. Dependence on $\ln (1 + z)$ is considered:

\begin{equation}\label{M_R_3}
\begin{split}
    \left<\ln R | M, z \right> = \mu_0 + 
    \mu_{M1} \ln\left( \frac{M}{M_0} \right) +
    \mu_{M2} (\ln\left(\frac{M}{M_0} \right))^2 +\\
    + \mu_{z1} \ln\left(\frac{1 + z}{1 + z_0}\right) +
    \mu_{z2} (\ln\left(\frac{1 + z}{1 + z_0}\right))^2 +\\
    + \mu_{Mz} \ln\left(\frac{1 + z}{1 + z_0}\right)\ln\left(\frac{M}{M_0} \right);
\end{split}
\end{equation}

$$\textbf{p}_{2} = \{\mu_0, \mu_{M1}, \mu_{M2}, \mu_{Z1}, \mu_{Z2}, \mu_{MZ}, \sigma_0, \sigma_{M1}, \sigma_{M2}, \sigma_{Z1}, \sigma_{Z2}, \sigma_{MZ} \}.$$

$\textbf{p}_{2}$ is the vector with the quadratic model parameters.


In [ ]:
ext_ln1pz, fit_ext = fm.run_fit('ext_ln1pz')

In [ ]:
lnR_mean_ext_ln1pz = fm.get_mean_model(ext_ln1pz,  np.log(halos_mean['mass']), halos_mean['redshift'])
lnR_std_ext_ln1pz = fm.get_std_model(ext_ln1pz,  np.log(halos_mean['mass']), halos_mean['redshift'])


### Plot config

In [ ]:
def Plot_model(model, h_mean, h_std, mean):
    
    fig = plt.figure(figsize=(16,4))
#     gs = gridspec.GridSpec(nrows=2, ncols=2, width_ratios=[1, 1], height_ratios=[2.5, 1], wspace=0.4)
    
#     ax1 = fig.add_subplot(gs[0, :], projection='3d')
#     ax2 = fig.add_subplot(gs[1, 0])
#     ax3 = fig.add_subplot(gs[1, 1])
    ax1 = fig.add_subplot(1,3,1, projection='3d')
    ax2 = fig.add_subplot(1,3,2)
    ax3 = fig.add_subplot(1,3,3)

    xs = h_mean["redshift"]
    ys = np.log(h_mean["mass"])
    zs = model

    lnR_mean = np.log(h_mean["richness"])
    std_mean = h_std

    
    #Mu plot config ---------------------------------------
    if mean == True:
        #lnR model, z_mean, y_mean
        
        lb_model = '<$Ln\lambda$| M, z>'
        lb_binned = 'Mean $\ln \lambda_i$'
        m = lnR_mean
        
        if f'{model}' == f'{lnR_mean_ascaso}': 
            #fig.suptitle('Modelo de Ascaso')
            fig.suptitle('$<\ln\lambda| M, z>_{LINEAR}$')
        elif f'{model}' == f'{lnR_mean_ext_ln1pz}': 
            fig.suptitle('$<\ln\lambda| M, z>_{QUADRATIC}$')
        elif f'{model}' == f'{lnR_mean_ext_z}': 
            fig.suptitle('Modelo Estendido (z)')
        else: 
            pass
    
    #Std plot config ---------------------------------------
    else:
        
        lb_model = '$\sigma_{\ln \lambda}$'
        lb_binned = '$\sigma^{i}$'
        m = std_mean
        
        if f'{model}' == f'{lnR_std_ascaso}': 
            fig.suptitle('Linear Model', size=16)
        elif f'{model}' == f'{lnR_std_ext_ln1pz}': 
            fig.suptitle('Quadratic Model(ln(1 + z))', size=16)
        elif f'{model}' == f'{lnR_std_ext_z}': 
            fig.suptitle('Quadratic Model (z)', size=16)
        else: 
            pass
        
    #-------------------------------------------------------
    p1 =ax1.scatter(xs, ys, zs, c=zs, cmap='cool')
    ax1.set_xlabel('z')
    ax1.set_ylabel('lnM')
    fig.colorbar(p1, ax=ax1, label= lb_model)

    ax2.scatter(ys, m, c='k', s=2.0, label=lb_binned)
    p2 = ax2.scatter(ys, zs , c= xs, s=2.0, cmap='cool')
    ax2.set_xlabel('lnM')
    ax2.set_ylabel(lb_model)
    fig.colorbar(p2, ax=ax2, label='z')
    ax2.legend()

    ax3.scatter(xs, m, c='k', s=2.0, label=lb_binned)
    p3 = ax3.scatter(xs, zs , c=ys, s=2.0, cmap='cool') 
    ax3.set_xlabel('z')
    ax3.set_ylabel(lb_model)
    fig.colorbar(p3, ax=ax3, label='lnM')
    ax3.legend()         
    plt.show()



### Mean

In [ ]:
Plot_model(lnR_mean_ascaso, halos_mean, halos_std, mean = True) #Ascaso
Plot_model(lnR_mean_ext_ln1pz, halos_mean, halos_std, mean = True) #Extended (ln(1 + z)) 
# Plot_model(lnR_mean_ext_z, data_test, mean = True)

plt.show()

### Standard deviation

In [ ]:
Plot_model(lnR_std_ascaso, halos_mean, halos_std, mean=False) #Ascaso
Plot_model(lnR_std_ext_ln1pz, halos_mean, halos_std, mean=False) #Extended (ln(1 + z)) 
#Plot_model_std(lnR_std_ext_z, data_train)
plt.show()



In [ ]:
# print(stats.skew(ext_std), stats.kurtosis(ext_std))
# plt.hist(ext_std, bins = 10)

## Cross Validation

- Relative error:

\begin{equation}
Rel\_err = \frac{\left<\ln R_i | M_i, z_i \right> - {\mu^{i}}_{lnR}}{{\mu^{i}}_{lnR}}
\end{equation}

- Mean Square Error (MSE):

\begin{equation}
MSE = \frac{1}{N} \sum^{N} ({\mu^{i}}_{lnR} - \left<\ln R_i | M_i, z_i \right> )^2
\end{equation}

- Root Mean Square Error (RMSE):

\begin{equation}
RMSE = \sqrt{MSE}
\end{equation}


### Mean Relative Error

In [ ]:
bd_test = BinningData(data_test)
bin_f_test= bd_test.get_mean_bd(b_z, b_m)

In [ ]:
# MEAN 
halos_mean = bin_f_test[0]

lnR_test = np.log(halos_mean["richness"])
lnM_test = np.log(halos_mean["mass"])
z_test = halos_mean["redshift"]

ascaso_mean_valid = np.array( [ascaso.get_mean_richness(lnM_test[i], z_test[i]) for i in range(len(lnR_test))])
ext_ln1pz_mean_valid = np.array([ext_ln1pz.get_mean_richness(lnM_test[i], z_test[i]) for i in range(len(lnR_test))])

In [ ]:
#Relative Error: Mean

#mean relative error
re_mean_ascaso = (ascaso_mean_valid - lnR_test)/ lnR_test
re_mean_ext_ln1pz = (ext_ln1pz_mean_valid - lnR_test)/ lnR_test

#Plot
plt.scatter(ascaso_mean_valid, re_mean_ascaso, c='#a0d995', alpha=0.7, label='Ascaso')
# plt.scatter(ext_ln1pz_mean_valid, re_mean_ext_ln1pz, c='#b186f1', alpha=0.7, label='Modelo Estendido (ln(1 + z))')
plt.xlabel('Predição', size=14)
plt.ylabel('Erro Relativo', size=14)
#plt.yscale('log')
plt.axhline(y=0.0, color='black', linestyle='dotted')

plt.title('Erro relativo da média', size=14)
plt.legend()
plt.show()


In [ ]:
#Relative Error histogram

fig = plt.figure(figsize=(10,3))
ax1=plt.subplot(121)
ax2=plt.subplot(122)

ax1.hist(re_mean_ascaso, bins=70, color='#a0d995')
ax1.set_title('Ascaso')

ax2.hist(re_mean_ext_ln1pz, bins=70, color='#b186f1')
ax2.set_title('Modelo Estendido (ln(1 + z))')

plt.show()

print(f" Ascaso: p-value: {stats.kstest(re_mean_ascaso, stats.norm.cdf, args=(np.mean(re_mean_ascaso), np.std(re_mean_ascaso))).pvalue}")
print(f" Extended: p-value: {stats.kstest(re_mean_ext_ln1pz, stats.norm.cdf, args=(np.mean(re_mean_ext_ln1pz), np.std(re_mean_ext_ln1pz))).pvalue}")



### Mean MSE and RMSE

In [ ]:
# Mean Square Error(MSE)
# Root Mean Square Error(RMSE)

MSE_ascaso = sum((lnR_test - ascaso_mean_valid) ** 2) / len(ascaso_mean_valid)
RMSE_ascaso = np.sqrt(MSE_ascaso)

MSE_ext_ln1pz = sum((lnR_test - ext_ln1pz_mean_valid) ** 2) / len(ext_ln1pz_mean_valid)
RMSE_ext_ln1pz = np.sqrt(MSE_ext_ln1pz )

print(f"Mean MSE and RMSE\n\nAscaso: \tMSE: {MSE_ascaso: .3f},\t RMSE:{RMSE_ascaso: .3f}")
print(f"Extended:\tMSE: {MSE_ext_ln1pz: .3f},\t RMSE:{RMSE_ext_ln1pz: .3f}\n")

### Mean MAPE

In [ ]:
print(f"MAPE:STD\n\nMAPE (Ascaso): {mean_absolute_percentage_error(lnR_test, ascaso_mean_valid) * 100: .2f}%") 
print(f"MAPE (Extended): {mean_absolute_percentage_error(lnR_test, ext_ln1pz_mean_valid) * 100: .2f}%") 

### Std Relative Error

In [ ]:
bd_test = BinningData(data_test)
bin_f_test= bd_test.get_mean_bd(b_z, b_m)

lnR_test_std = bin_f_test[4]

ascaso_std_valid = np.array([ascaso.get_std_richness(lnM_test[i], z_test[i]) for i in range(len(lnR_test_std))])
ext_ln1pz_std_valid = np.array([ext_ln1pz.get_std_richness(lnM_test[i], z_test[i]) for i in range(len(lnR_test_std))])


In [ ]:
abs_err = (lnR_test - ascaso_mean_valid) / lnR_test_std

plt.scatter(lnM_test, abs_err)

In [ ]:
# STD 
#Relative Error: std
a_err = (ascaso_std_valid - lnR_test_std)/ lnR_test_std
e_err = (ext_ln1pz_std_valid - lnR_test_std)/ lnR_test_std

#Plot
plt.scatter(ascaso_std_valid, a_err, c='#a0d995', alpha=0.7, label='Ascaso')
plt.scatter(ext_ln1pz_std_valid, e_err, c='#b186f1', alpha=0.7, label='Modelo Estendido (ln(1 + z))')

plt.xlabel('Predição', size=14)
plt.ylabel('Erro Relativo', size=14)
plt.axhline(y=0.0, color='black', linestyle='dotted')
#plt.title('Erro relativo do desvio padrão (< 10)', size=16)
#plt.yscale('log')
plt.legend()
plt.show()

In [ ]:
#Relative Error histogram

# fig = plt.figure(figsize=(10,3))
# ax1=plt.subplot(121)
# ax2=plt.subplot(122)

# ax1.hist(a_err, bins=70, density=True, color='#a0d995')
# ax1.set_title('Ascaso')

# ax2.hist(e_err, bins=70,density=True, color='#b186f1')
# ax2.set_title('Modelo Estendido (ln(1 + z))')

# plt.show()

# print(f" Ascaso: p-value: {stats.kstest(a_err, stats.norm.cdf, args=(np.mean(a_err), np.std(a_err))).pvalue}")
# print(f" Extended: p-value: {stats.kstest(e_err, stats.norm.cdf, args=(np.mean(e_err), np.std(e_err))).pvalue}")

### Std MSE and RMSE

In [ ]:
MSE_ascaso_std = sum((lnR_test_std - ascaso_std_valid) ** 2) / len(ascaso_std_valid)
RMSE_ascaso_std = np.sqrt(MSE_ascaso_std)

MSE_ext_ln1pz_std = sum((lnR_test_std - ext_ln1pz_std_valid) ** 2) / len(ext_ln1pz_std_valid)
RMSE_ext_ln1pz_std = np.sqrt(MSE_ext_ln1pz_std )

print(f"Std MSE and RMSE\n\nAscaso: \tMSE: {MSE_ascaso_std: .3f},\t RMSE:{RMSE_ascaso_std: .3f}")
print(f"Extended:\tMSE: {MSE_ext_ln1pz_std: .3f},\t RMSE:{RMSE_ext_ln1pz_std: .3f}\n")

### Std MAPE

In [ ]:
print(f"MAPE:STD\n\nMAPE (Ascaso): {mean_absolute_percentage_error(lnR_test_std, ascaso_std_valid) * 100: .2f}%") 
print(f"MAPE (Extended): {mean_absolute_percentage_error(lnR_test_std, ext_ln1pz_std_valid) * 100: .2f}%") 